In [1]:
from omegaconf import OmegaConf
import numpy as np
import pickle as pkl
import torch
import sys
sys.path.append("/dreambig/qingyi/stable-diffusion")

from dataloader import image_loader
from setup import *
from scripts.img2img import load_model_from_config

%load_ext autoreload
%autoreload 2

In [2]:
# data loaders
zoomlevel = 'zoom15'
data_version = '1571'
sampling = 'stratified'
batch_size = 4
num_workers = 8
image_size = 256

train_loader, test_loader = image_loader(image_dir+zoomlevel+"/", data_dir, batch_size, 
         num_workers, 
         image_size, 
         data_version=data_version, 
         sampling=sampling, 
         augment=False)

26707 images in dataset
4713 images in dataset


In [4]:
# ckpt = '/dreambig/qingyi/stable-diffusion/logs/2023-06-21T17-21-27_autoencoder_kl_32x32x4/checkpoints/epoch=000002.ckpt'
ckpt = '/dreambig/qingyi/stable-diffusion/logs/2023-06-24T19-02-38_autoencoder_kl_8x8x64/checkpoints/epoch=000020.ckpt'
# ckpt = '/dreambig/qingyi/stable-diffusion/models/first_stage_models/kl-f32/model.ckpt'
config = OmegaConf.load('/dreambig/qingyi/stable-diffusion/models/first_stage_models/kl-f32/config.yaml')
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = load_model_from_config(config, ckpt, verbose=True)


Loading model from /dreambig/qingyi/stable-diffusion/logs/2023-06-24T19-02-38_autoencoder_kl_8x8x64/checkpoints/epoch=000020.ckpt
Global Step: 150066
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 64, 8, 8) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
making attention of type 'vanilla' with 512 in_channels
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [5]:
ct = []
encoder_output = []
im = []

for step, data in enumerate(test_loader):
    data1 = data[1].permute(0, 3, 1, 2).to(device)
    encoder_output += [model.encode(data1).mode().cpu().detach().numpy()]
    ct += [s[s.rindex("/")+1: s.rindex("_")]for s in data[0]]
#     encoder_output += [model.encode(data1).cpu().detach().numpy()]
    im += data[0]
    if step % 10 == 0:
        print(step, end='\t')
        
for step, data in enumerate(train_loader):
    data1 = data[1].permute(0, 3, 1, 2).to(device)
    encoder_output += [model.encode(data1).mode().cpu().detach().numpy()]
    ct += [s[s.rindex("/")+1: s.rindex("_")]for s in data[0]]
#     encoder_output += [model.encode(data1).cpu().detach().numpy()]
    im += data[0]
    if step % 10 == 0:
        print(step, end='\t')

0	10	20	30	40	50	60	70	80	90	100	110	120	130	140	150	160	170	180	190	200	210	220	230	240	250	260	270	280	290	300	310	320	330	340	350	360	370	380	390	400	410	420	430	440	450	460	470	480	490	500	510	520	530	540	550	560	570	580	590	600	610	620	630	640	650	660	670	680	690	700	710	720	730	740	750	760	770	780	790	800	810	820	830	840	850	860	870	880	890	900	910	920	930	940	950	960	970	980	990	1000	1010	1020	1030	1040	1050	1060	1070	1080	1090	1100	1110	1120	1130	1140	1150	1160	1170	0	10	20	30	40	50	60	70	80	90	100	110	120	130	140	150	160	170	180	190	200	210	220	230	240	250	260	270	280	290	300	310	320	330	340	350	360	370	380	390	400	410	420	430	440	450	460	470	480	490	500	510	520	530	540	550	560	570	580	590	600	610	620	630	640	650	660	670	680	690	700	710	720	730	740	750	760	770	780	790	800	810	820	830	840	850	860	870	880	890	900	910	920	930	940	950	960	970	980	990	1000	1010	1020	1030	1040	1050	1060	1070	1080	1090	1100	1110	1120	1130	1140	1150	1160	1170	1180	1190	1200	1210	1220	1230	1240	1250	12

In [6]:
encoder_output = np.vstack(encoder_output)  

In [8]:
encoder_output.shape

(31420, 64, 8, 8)

In [7]:
with open(proj_dir+"latent_space/SDAE_"+zoomlevel+"_kl-f32_ep20.pkl", "wb") as f:

    pkl.dump(encoder_output, f)
    pkl.dump(im, f)
    pkl.dump(ct, f)
            